In [1]:
pip install icecream


Note: you may need to restart the kernel to use updated packages.


In [119]:
import requests ## to capture content from web pages
from bs4 import BeautifulSoup ## to parse our scraped data
import pandas as pd ## to easily export our data to dataframes/CSVs
from icecream import ic ## easily debug
from pprint import pprint as pp ## to prettify our printouts
import itertools ## to flatten lists
from random import randrange ## to create a range of numbers
import time # for timer
import json ## to work with JSON data

In [107]:
## Clean Heat, Commercial Tennant Green Contractors, Commercial/Industrial Solar Installers:
## The three contractor types have totally different target site names, so it made sense 
## to just write them out in a list.

urls = ["https://portal.nyserda.ny.gov/FindAContractorNYSCleanHeat", "https://nyserda-site.force.com/FindAContractorComTenant",
        "https://nyserda-site.secure.force.com/FindAContractorNYSun?programname=NY-Sun%20-%20Commercial%20&%20Industrial"]


In [108]:
## Scrape all the tables on the urls into a single list of dfs

all_base_dfs = []

for url in urls:
    response = requests.get(url)
#    ic(response.status_code)
    df_list = pd.read_html(response.text)
    df = df_list[0]
    all_base_dfs.append(df)


In [113]:
## CLEAN HEAT CONTRACTORS

## Each df is different enough to require its own regex for organizing the columns.

## Although the data imports easily as a table, the company info is all jammed together in a single column.
## Target each element using regex and assign it to its own column.
## I would now use re.compile, but I started this before I knew that code! For the sake of time, I'm going w what I got:

df_clean_heat = all_base_dfs[0]

df_clean_heat['Contact_Name'] = df_clean_heat['Company'].str.extract(r'(\s\s\D+)')
## trim off the extra ( included in contact name
df_clean_heat['Contact_Name'] = df_clean_heat['Contact_Name'].str[:-1]

df_clean_heat['Company_Name']= df_clean_heat['Company'].str.extract(r'(^\d*?\D+\s\s)')

df_clean_heat['Phone_Number'] = df_clean_heat['Company'].str.extract(r'(\W\d{3}\W\s\d{3}-\d{4})')

df_clean_heat['Email'] = df_clean_heat['Company'].str.extract(r'(\w+@\w+\D+)')
## trim off the 4 phone number digits that got included in the email
df_clean_heat['Email'] = df_clean_heat['Email'].str[4:]

## The addresses import jammed together without spaces which geopy cannot read. 
## Separate out each address element and connect them with a space.

addressln1_city_state_pat = re.compile(""" (\d+\s\w+\s[\w+\s]*[A-Z][a-z]+) # capture address line 1
                                     ([A-Z][a-z]+[\s\,\w+]*\d{5}) # capture city, state, zip code
                                     """, 
                                        re.M | re.X)
## Clean heat:
df_clean_heat = all_base_dfs[0]
useable_address = []
for company in df_clean_heat["Company"]:
    address_compile = addressln1_city_state_pat.findall(company)
    useable_address.append(address_compile)    
df_clean_heat['Useable_Address'] = useable_address



In [1]:
## Create a function to create CSV files:

def create_csv(df_name, file_name):
    '''
    Export your dataframe as a csv
    argument 1 = your dataframe name
    argument 2 = your file name as a string. Must include .csv
    For example "my_data.csv"
    '''
    df_name.to_csv(file_name, encoding='utf-8', index=False )
    print(f"{file_name} is in your local folder!")

In [71]:
## CLEAN HEAT CONTRACTORS

## The regex that isolated the majority of the addresses will be easier to deal with in Excel. 
## Convert the df to csv and clean the remaining addresses from there. ()

create_csv(df_clean_heat, "clean-heat.csv")

In [74]:
## CLEAN HEAT CONTRACTORS

## Once the data is cleaned, import the csv file back into pandas:

cleaned_clean_heat_df = pd.read_csv (r'cleaned-clean-heat.csv')


In [ ]:
## The other two dfs are much shorter and it's just not worth it to figure out regex for their addresses.
## Send them to csv files and manually deconstruct their addresses
df_com_ten = all_base_dfs[1]
df_solar = all_base_dfs[2]

create_csv(df_com_ten, "com-ten.csv")
create_csv(df_solar,"solar.csv")


In [ ]:
## Once cleaned, import the csv files above back into pandas

cleaned_com_ten_df = pd.read_csv (r'cleaned-com-ten.csv')
cleaned_solar_df = pd.read_csv (r'cleaned-solar.csv')


In [ ]:
## Multi-Family Building Clean Contractors

## This NYSERDA site is totally different from the previous pages and can be imported through the json package

url = "https://www.nyserda.ny.gov/rapi/contractorfinderapi/getmppcontractors?id=b0dcf247-4ea2-4fc4-a3ee-380c34e6faed&counties=&companyname="

## Using the json site found in the "Network" tab of the inspect tab, scrape the page:

response = requests.get(url)
content = response.json() 
target = content.get("Contractors")
multi_fam_df = pd.DataFrame(target)

## The address comes in four parts. Combine them:

multi_fam_df["Address"] = multi_fam_df['Street1EB'] + " " + multi_fam_df['CityEB'] + ' ' + 
                            multi_fam_df['StateEB'] + ' ' + multi_fam_df['Zip5NC']


In [ ]:
## Now, we're ready to start with geocoding. 
## GIS requires the csv files to remain separate, so don't compile

In [114]:
pip install geopandas


Note: you may need to restart the kernel to use updated packages.


In [115]:
pip install geopy


Note: you may need to restart the kernel to use updated packages.


In [116]:
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter


In [120]:
## Create a geocoding function and run each df through it. 
##Then download the resulting geocoded dfs to csv files to be used in QGIS


def geocoder(df, address_column_name):
    '''
    Gives the geocode (lat, long) of an address
    df = some dataframe with a column for addresses
    address_column_name = where in the df the addresses are held
    '''
    # Nominatim geocoder for OpenStreetMap data with RateLimiter
    geocoder = RateLimiter(Nominatim(user_agent='tutorial').geocode, min_delay_seconds=1)
    df['Location'] = df[address_column_name].apply(geocoder)

    # add latitude and longitude to dataframe
    df['Latitude'] = df['Location'].apply(lambda loc: loc.latitude if loc else None)
    df['Longitude'] = df['Location'].apply(lambda loc: loc.longitude if loc else None)
    
geocoder(cleaned_clean_heat_df, 'Useable_Address')
geocoder(cleaned_com_ten_df,'Useable_Address')
geocoder(cleaned_solar_df,'Useable_Address')
geocoder(multi_fam_df,'Address')

## With geocodes in hand, download this df to a csv file to manually check the addresses geopy failed to geocode

create_csv(cleaned_clean_heat_df, "clean-clean-clean-CLEAN-clean-heat.csv")
create_csv(cleaned_com_ten_df, "com-ten-lat-long.csv")
create_csv(cleaned_solar_df, "solar-lat-long.csv")
create_csv(multi_fam_df,"multi-family.csv")

RateLimiter caught an error, retrying (0/2 tries). Called with (*('6711 St Hwy 11 Canton, NY 13617',), **{}).
Traceback (most recent call last):
  File "/Users/slabowitz/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py", line 445, in _make_request
    six.raise_from(e, None)
  File "<string>", line 3, in raise_from
  File "/Users/slabowitz/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py", line 440, in _make_request
    httplib_response = conn.getresponse()
  File "/Users/slabowitz/opt/anaconda3/lib/python3.8/http/client.py", line 1347, in getresponse
    response.begin()
  File "/Users/slabowitz/opt/anaconda3/lib/python3.8/http/client.py", line 307, in begin
    version, status, reason = self._read_status()
  File "/Users/slabowitz/opt/anaconda3/lib/python3.8/http/client.py", line 268, in _read_status
    line = str(self.fp.readline(_MAXLINE + 1), "iso-8859-1")
  File "/Users/slabowitz/opt/anaconda3/lib/python3.8/socket.py", line 669, in readinto


In [121]:
cleaned_clean_heat_df

,Company,Specialties,Counties Served,Utilities,Company Description and Case Studies,Contact_Name,Company_Name,Phone_Number,Email,Useable_Address,Location,Latitude,Longitude
0,"Reiner Group, Inc. Anthony Gasparovich(201) 7...",ASHP Installer,Rockland,Orange and Rockland,NaN,Anthony Gasparovich,"Reiner Group, Inc.",(201) 794-3700,tgasparovich@reinerac.com,"11-07 River Rd, Fair Lawn, NJ 07410","(River Road, Fair Lawn, Bergen County, New Jer...",40.926704,-74.129747
1,Perfection Contracting Inc Sam Qualagari(973)...,"ASHP Installer, GSHP Designer <300,000 Btu/h, ...","Essex, Monroe, Orange, Rockland, Warren, Westc...","Central Hudson Gas & Electric, Consolidated Ed...",NaN,Sam Qualagari,Perfection Contracting Inc,(973) 579-0182,sam@perfectioncontracting.com,"1 Camre Dr, Newton, New Jersey 07860",None,NaN,NaN
2,"DWC Mechanical, Inc. Robin Cloy(716) 891-0350...",ASHP Installer,"Cattaraugus, Chautauqua, Erie, Monroe, Niagara...","National Grid, New York State Electric and Gas...",NaN,Robin Cloy,"DWC Mechanical, Inc.",(716) 891-0350,rcloy@dwcmechanical.com,"100 John Glenn Dr, Amherst, New York 14228","(100, John Glenn Drive, Audubon Industrial Par...",43.031244,-78.806116
3,Heatwave Heating and Cooling Robin Cloy(716) ...,ASHP Installer,"Cattaraugus, Chautauqua, Erie, Monroe, Niagara...","National Grid, New York State Electric and Gas...",NaN,Robin Cloy,Heatwave Heating and Cooling,(716) 891-9283,rcloy@heatwavewny.com,"100 John Glenn Dr, Amherst, New York 14228","(100, John Glenn Drive, Audubon Industrial Par...",43.031244,-78.806116
4,Good-Will Mechanical Corp. Jenny Li(718) 486-...,ASHP Installer,"Bronx, New York, Queens, Westchester, Kings",Consolidated Edison,NaN,Jenny Li,Good-Will Mechanical Corp.,(718) 486-9000,jli@goodwillmech.com,"1000 Grand St, Brooklyn, NY 11211","(Grand Street, Queens, Brooklyn, Kings County,...",40.716591,-73.922352
...,...,...,...,...,...,...,...,...,...,...,...,...,...
511,KL Cooling and Heating Kalmen Landau(718) 435...,ASHP Installer,"Bronx, Delaware, Monroe, New York, Queens, Roc...",Consolidated Edison,NaN,Kalmen Landau,KL Cooling and Heating,(718) 435-3991,kalmen@klnyac.com,"5308 13th Ave Brooklyn, NY 11219","(5308, 13th Avenue, Borough Park, Brooklyn, Ki...",40.632618,-73.994837
512,With Pride Air Conditioning and Heating Joani...,ASHP Installer,"Bronx, New York, Westchester, Nassau, Kings",Consolidated Edison,NaN,Joanie Barton,With Pride Air Conditioning and Heating,(516) 731-2573,joanie@withpridehvac.net,"90 Verdi St. Farmingdale, New York 11735","(90, Verdi Street, Suffolk County, New York, 1...",40.713446,-73.420930
513,"WK Mechanical, Inc. Stephen Conforti(845) 782...",ASHP Installer,"Dutchess, Orange, Ulster, Sullivan","Central Hudson Gas & Electric, New York State ...",NaN,Stephen Conforti,"WK Mechanical, Inc.",(845) 782-8133,sconforti@wkmechanical.com,"546 State Route 17M Monroe, NY 10950",None,NaN,NaN
514,Wood Well Drilling Darren Wood(518) 483-2932w...,GSHP Driller,"Clinton, Essex, Franklin, St. Lawrence","National Grid, New York State Electric and Gas",NaN,Darren Wood,Wood Well Drilling,(518) 483-2932,wooddrilling@yahoo.com,"23285 State Route 11B Bangor, NY 12966",None,NaN,NaN
